In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline

In [2]:
# Import DataFrame 
main_df = pd.read_csv('Rates_MO.csv')
main_df.columns

Index(['Unnamed: 0', 'observation_date', 'C&I_DELNQ', 'CCARD_CO',
       'CCARD_DELNQ', 'CORP_DEBT_NET_WORTH', 'CORP_SAVINGS_LEVEL', 'CRE_CO',
       'CRE_DELNQ', 'GDP', 'Homeowner_Vacancy_rate', 'Household_DBT_Inc',
       'Mortgage_CO', 'Mortgage_DELNQ', 'Rental_Vacancy_Rate',
       'Consumer_Confidence', 'FEDFUNDS', 'Manufacturing_Confidence',
       'SAVINGS_RATE_MO', 'UNRATE', 'C&I_CO'],
      dtype='object')

In [3]:
# Create credit card dataset
ccard_df = main_df[['observation_date', 'CCARD_CO', 'CCARD_DELNQ', 'GDP', 'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS', 'SAVINGS_RATE_MO', 'UNRATE']]

In [4]:
ccard_df.dtypes

observation_date        object
CCARD_CO               float64
CCARD_DELNQ            float64
GDP                    float64
Household_DBT_Inc      float64
Consumer_Confidence    float64
FEDFUNDS               float64
SAVINGS_RATE_MO        float64
UNRATE                 float64
dtype: object

In [5]:
ccard_bin_df = ccard_df.copy()

In [6]:
#ccard_df['CCARD_CO']=ccard_df['CCARD_CO'].astype(int)

In [7]:
# Find Minimum and Maximum values to determine number of bins
a=ccard_df['CCARD_CO'].max()
b=ccard_df['CCARD_CO'].min()
c=ccard_df['CCARD_CO'].count()
print(a)
print(b)
print(c)

10.51
1.66
390


In [8]:
# Binning the data for classification Question: 
# Should we be using pd cut to get more bins? 
ccard_bin_df["CCARD_CO_BIN"] = pd.qcut(ccard_df['CCARD_CO'],4, labels= [1, 2, 3, 4])
ccard_bin_df["CCARD_DELNQ_BIN"] = pd.qcut(ccard_df['CCARD_DELNQ'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["GDP_BIN"] = pd.qcut(ccard_df['GDP'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["Household_DBT_Inc_BIN"] = pd.qcut(ccard_df['Household_DBT_Inc'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["Consumer_Confidence_BIN"] = pd.qcut(ccard_df['Consumer_Confidence'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["FEDFUNDS_BIN"] = pd.qcut(ccard_df['FEDFUNDS'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["SAVINGS_RATE_MO_BIN"] = pd.qcut(ccard_df['SAVINGS_RATE_MO'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])
ccard_bin_df["UNRATE_BIN"] = pd.qcut(ccard_df['UNRATE'],4, labels= ['low', 'medium-low', 'medium-high', 'high'])


In [9]:
ccard_bin_df.head()

,observation_date,CCARD_CO,CCARD_DELNQ,GDP,Household_DBT_Inc,Consumer_Confidence,FEDFUNDS,SAVINGS_RATE_MO,UNRATE,CCARD_CO_BIN,CCARD_DELNQ_BIN,GDP_BIN,Household_DBT_Inc_BIN,Consumer_Confidence_BIN,FEDFUNDS_BIN,SAVINGS_RATE_MO_BIN,UNRATE_BIN
0,1991-01-01,4.16,5.26,-1.9,11.578032,66.8,6.91,9.4,6.4,3,high,low,medium-high,low,high,high,medium-high
1,1991-02-01,4.16,5.26,-1.9,11.578032,70.4,6.25,9.0,6.6,3,high,low,medium-high,low,high,high,medium-high
2,1991-03-01,4.16,5.26,-1.9,11.578032,87.7,6.12,8.1,6.8,3,high,low,medium-high,medium-low,high,high,high
3,1991-04-01,4.60,5.48,3.2,11.434237,81.8,5.91,8.7,6.7,3,high,medium-high,medium-high,medium-low,high,high,medium-high
4,1991-05-01,4.60,5.48,3.2,11.434237,78.3,5.78,8.5,6.9,3,high,medium-high,medium-high,medium-low,high,high,high


In [10]:
# Seperate the y and X variables
y = ccard_bin_df["CCARD_CO_BIN"]
X_bin = ccard_bin_df.drop(columns=["CCARD_CO", "observation_date", "CCARD_DELNQ", "GDP", "Household_DBT_Inc", "Consumer_Confidence", "FEDFUNDS", "SAVINGS_RATE_MO", "UNRATE", "CCARD_CO_BIN"])
X_bin.columns

Index(['CCARD_DELNQ_BIN', 'GDP_BIN', 'Household_DBT_Inc_BIN',
       'Consumer_Confidence_BIN', 'FEDFUNDS_BIN', 'SAVINGS_RATE_MO_BIN',
       'UNRATE_BIN'],
      dtype='object')

In [11]:
# Turn data into dummies
X_bin =pd.get_dummies(X_bin)
X = pd.concat([y, X_bin])
X

,0,CCARD_DELNQ_BIN_low,CCARD_DELNQ_BIN_medium-low,CCARD_DELNQ_BIN_medium-high,CCARD_DELNQ_BIN_high,GDP_BIN_low,GDP_BIN_medium-low,GDP_BIN_medium-high,GDP_BIN_high,Household_DBT_Inc_BIN_low,...,FEDFUNDS_BIN_medium-high,FEDFUNDS_BIN_high,SAVINGS_RATE_MO_BIN_low,SAVINGS_RATE_MO_BIN_medium-low,SAVINGS_RATE_MO_BIN_medium-high,SAVINGS_RATE_MO_BIN_high,UNRATE_BIN_low,UNRATE_BIN_medium-low,UNRATE_BIN_medium-high,UNRATE_BIN_high
0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
386,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
387,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
388,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X_bin, y, random_state=78)

In [13]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
len(X_train_scaled)

292

In [15]:
# Create a random forest classifier
cc_model = RandomForestClassifier(n_estimators=100, random_state=78)

In [16]:
# Fitting the model
cc_model = cc_model.fit(X_train_scaled, y_train)

In [17]:
# Making predictions using the testing data
predictions = cc_model.predict(X_test_scaled)

In [18]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Low", "Actual Medium-Low", "Actual Medium-High", "High"], columns=["Predicted Low", "Predicted Medium-Low", "Predicted Medium-High", "Predicted High"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [19]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Low,Predicted Medium-Low,Predicted Medium-High,Predicted High
Actual Low,24,1,0,0
Actual Medium-Low,1,19,4,0
Actual Medium-High,0,0,21,3
High,0,1,1,23


Accuracy Score : 0.8877551020408163
Classification Report
              precision    recall  f1-score   support

           1       0.96      0.96      0.96        25
           2       0.90      0.79      0.84        24
           3       0.81      0.88      0.84        24
           4       0.88      0.92      0.90        25

    accuracy                           0.89        98
   macro avg       0.89      0.89      0.89        98
weighted avg       0.89      0.89      0.89        98



# Part 2: Logistic Regression Model

In [20]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [21]:
# Train the data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [22]:
# Predict outcomes for test data set
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
384,1,1
206,4,3
132,4,4
226,4,4
223,4,4
...,...,...
300,1,1
293,1,1
71,4,3
370,1,1


In [23]:
from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.8061224489795918

# Part 3: Deep Learning First Attempt: Categorized and Scaled

In [24]:
ccard_df.columns

Index(['observation_date', 'CCARD_CO', 'CCARD_DELNQ', 'GDP',
       'Household_DBT_Inc', 'Consumer_Confidence', 'FEDFUNDS',
       'SAVINGS_RATE_MO', 'UNRATE'],
      dtype='object')

In [25]:
# Seperate the y and X variables
# y = ccard_df["CCARD_CO"]
# X = ccard_df.drop(columns=["CCARD_CO", "observation_date"])
# X.columns

In [27]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X_bin, y, random_state=78)

In [28]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
print(X_train_scaled)

[[ 1.64201173 -0.55097317 -0.60372674 ... -0.57735027 -0.59317101
  -0.56152944]
 [-0.60900905 -0.55097317  1.6563785  ... -0.57735027  1.68585446
  -0.56152944]
 [-0.60900905 -0.55097317  1.6563785  ... -0.57735027 -0.59317101
   1.78085052]
 ...
 [-0.60900905 -0.55097317  1.6563785  ... -0.57735027 -0.59317101
  -0.56152944]
 [-0.60900905  1.81497043 -0.60372674 ... -0.57735027  1.68585446
  -0.56152944]
 [-0.60900905 -0.55097317  1.6563785  ...  1.73205081 -0.59317101
  -0.56152944]]


In [33]:
import tensorflow as tf

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled)
hidden_nodes_layers1 = 100
hidden_nodes_layers2 = 100
hidden_nodes_layers3 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=28, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               2900      
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 23201 (90.63 KB)
Trainable params: 23201 (90.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
10/10 [==============================] - 0s 803us/step - loss: -1.0768 - accuracy: 0.2260
Epoch 2/100
10/10 [==============================] - 0s 610us/step - loss: -6.4963 - accuracy: 0.2534
Epoch 3/100
10/10 [==============================] - 0s 632us/step - loss: -17.0823 - accuracy: 0.2534
Epoch 4/100
10/10 [==============================] - 0s 652us/step - loss: -39.5202 - accuracy: 0.2534
Epoch 5/100
10/10 [==============================] - 0s 537us/step - loss: -84.5766 - accuracy: 0.2534
Epoch 6/100
10/10 [==============================] - 0s 737us/step - loss: -173.9939 - accuracy: 0.2534
Epoch 7/100
10/10 [==============================] - 0s 559us/step - loss: -337.9931 - accuracy: 0.2534
Epoch 8/100
10/10 [==============================] - 0s 639us/step - loss: -595.7937 - accuracy: 0.2534
Epoch 9/100
10/10 [==============================] - 0s 733us/step - loss: -1025.2928 - accuracy: 0.2534
Epoch 10/100
10/10 [==============================] - 0s 794us/step - 

Epoch 77/100
10/10 [==============================] - 0s 770us/step - loss: -23045272.0000 - accuracy: 0.2534
Epoch 78/100
10/10 [==============================] - 0s 733us/step - loss: -24379924.0000 - accuracy: 0.2534
Epoch 79/100
10/10 [==============================] - 0s 777us/step - loss: -25730672.0000 - accuracy: 0.2534
Epoch 80/100
10/10 [==============================] - 0s 1ms/step - loss: -27091966.0000 - accuracy: 0.2534
Epoch 81/100
10/10 [==============================] - 0s 662us/step - loss: -28489252.0000 - accuracy: 0.2534
Epoch 82/100
10/10 [==============================] - 0s 790us/step - loss: -29913920.0000 - accuracy: 0.2534
Epoch 83/100
10/10 [==============================] - 0s 718us/step - loss: -31430638.0000 - accuracy: 0.2534
Epoch 84/100
10/10 [==============================] - 0s 759us/step - loss: -32968922.0000 - accuracy: 0.2534
Epoch 85/100
10/10 [==============================] - 0s 585us/step - loss: -34607996.0000 - accuracy: 0.2534
Epoch 86/100

# Deep Learning Second Attempt

In [ ]:
# Seperate the y and X variables
y = ccard_bin_df["CCARD_CO_BIN"]
X = ccard_bin_df.drop(columns=["CCARD_CO", "observation_date", "CCARD_DELNQ", "GDP", "Household_DBT_Inc", "Consumer_Confidence", "FEDFUNDS", "SAVINGS_RATE_MO", "UNRATE"])
X.columns

In [ ]:
# Turn data into dummies
X=pd.get_dummies(X)

X.head()

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaler
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled)

In [ ]:
X_train_scaled.dtype

In [ ]:
import tensorflow as tf

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled)
hidden_nodes_layers1 = 100
hidden_nodes_layers2 = 100
hidden_nodes_layers3 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=32, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layers3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_nn = nn.fit(X_train_scaled, y_train, epochs=100)